<a href="https://colab.research.google.com/github/Meridor6919/BeeClassification/blob/Functional-API/BeeClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import keras
from keras import layers
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [5]:
dataset = tfds.load('bee_dataset', split=['train'], as_supervised=True)

In [6]:
# Split between training data, validation data, test data
splitSize = (0.0, 0.765, 0.9, 1.0)

array = np.vstack(list(tfds.as_numpy(dataset[0])))

ranges = list([int(array.shape[0] * x) for x in splitSize])

def getDataset(ranges, idx):
    return (
        np.array(list(map(lambda x: x[0][:, :, 0], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['cooling_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['pollen_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['varroa_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['wasps_output'], array[ranges[idx]: ranges[idx + 1]])))
    )

# 0 - train, 1 - validation, 2 - test
images = [None] *3 
   
_input = keras.Input(shape=(300, 150, 1), name="img")
x = layers.Conv2D(32, (3, 3), activation='relu')(_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(64, (3, 3), activation='relu')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
_output = layers.Dense(4)(x)


# wszystkie dane na raz (z 4 parametrow robimy wektory czteroelementowe)
labels_all = [None] * 3
for i in range(3):
    helper = getDataset(ranges, i)
    labels_all[i] = np.column_stack((helper[1], helper[2], helper[3], helper[4]))

print(str(labels_all[0].shape))
print(str(labels_all[1].shape))

model_all = keras.Model(_input, _output, name="model_all")
# model_all.compile(optimizer=tf.keras.optimizers.Adadelta(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
# tf.keras.losses.CategoricalCrossentropy()
model_all.compile(optimizer=tf.keras.optimizers.Adadelta(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history_all = model_all.fit(images[0], labels_all[0], validation_data=(images[1], labels_all[1]), epochs=2)


print(max(history_all.history['accuracy']))

/usr/local/lib/python3.8/dist-packages/numpy/core/shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


(5729, 4)
(1012, 4)


NameError: ignored

Testing precision on red channel only

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/numpy/core/shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


Epoch 1/5
199/199 [==============================] - 18s 48ms/step - loss: 122595.2188 - accuracy: 0.8140 - val_loss: 0.4567 - val_accuracy: 0.8310
Epoch 2/5
199/199 [==============================] - 8s 42ms/step - loss: 0.4728 - accuracy: 0.8310 - val_loss: 0.4572 - val_accuracy: 0.8310
Epoch 3/5
199/199 [==============================] - 9s 44ms/step - loss: 0.4651 - accuracy: 0.8310 - val_loss: 0.5011 - val_accuracy: 0.8310
Epoch 4/5
199/199 [==============================] - 9s 43ms/step - loss: 0.4719 - accuracy: 0.8310 - val_loss: 0.4568 - val_accuracy: 0.8310
Epoch 5/5
199/199 [==============================] - 8s 42ms/step - loss: 0.4660 - accuracy: 0.8310 - val_loss: 0.4592 - val_accuracy: 0.8310
